# Habituation to weakly non-Gaussian odor backgrounds
Look at a case with three odors. Compare BioPCA and IBCM models for habituation and new odor detection. 

## Some notes on the setting
### Saturation function on IBCM neurons
Since the fluctuations are still fast compared to the IBCM neurons, we do not really need saturation functions to prevent numerical divergences (while still using a large time step). Also, the model converges fast enough that we don't need the Law and Cooper, 1994 modification. 

### Background process
We simulate a zero-mean Ornstein-Uhlenbeck process $\tilde{\nu}$, then we set the actual odor concentrations to be $\nu = \nu_0 + \tilde{\nu} + \epsilon \tilde{\nu}^2$. This ensures that the odor concentrations have a third moment of order $\epsilon$. More precisely, we find that if there are no correlations, we can treat each component $\nu_{\gamma}$ as a univariate case, and we then have a third moment of order $\epsilon$, with only lower-order corrections to the second moment and order $\epsilon$ corrections to the desired mean $\nu_0$:

$$ \langle \nu \rangle = \nu_0 + \epsilon \sigma^2 $$
$$ \langle (\nu - \langle \nu \rangle)^2 \rangle = \sigma^2 + 2 \epsilon^2 \sigma^4 $$
$$ \langle (\nu - \langle \nu \rangle)^3 \rangle = 6 \epsilon \sigma^4 + 8 \epsilon^3 \sigma^6 $$

For multiple odors, $\tilde{\nu}$ is a multivariate O-U process as defined in Gardiner's Handbook, and the term $\epsilon \tilde{\nu}^2$ is computed element-wise. But in practice, we only consider independent odors, each can thus be thought of as a scalar O-U process. 


## Imports

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
    ibcm_respond_new_odors
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    fixedpoint_thirdmoment_perturbtheory, 
    ibcm_fixedpoint_w_thirdmoment, 
    jacobian_fixedpoint_thirdmoment, 
    ibcm_all_largest_eigenvalues, 
    lambda_pca_equivalent
)
from modelfcts.biopca import (
    integrate_inhib_ifpsp_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor, 
    compute_optimal_matrices
)
from modelfcts.checktools import (
    check_conc_samples_powerlaw_exp1,
    compute_pca_meankept, 
    compute_projector_series, 
    analyze_pca_learning
)
from modelfcts.backgrounds import (
    update_thirdmoment_kinputs, 
    sample_ss_distrib_thirdmoment, 
    sample_background_thirdmoment, 
    decompose_nonorthogonal_basis, 
    generate_odorant
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
    tags_list_to_csr_matrix
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gammas_sums, 
    plot_cbars_gamma_series, 
    plot_3d_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard
import itertools

In [ ]:
from utils.metrics import l2_norm, l1_norm, linf_norm, cosine_dist

def distance_panel_target(mixes, target):
    """ Compute a panel of distances between the pure (target) new odor and mixtures 
    (which can be without inhibition, with average inhibition, IBCM inhibition, etc.). 
    
    Four distances included, in order: l2, l1, linf, cosine_dist
    
    Args:
        mixes (np.ndarray): mixtures of odors to compute distance from target, 
            the last axis should have the size of target, 
            while other axes are arbitrary.  
        target (np.1darray): target odor vector, same length as
            last axis of mixes. 
    Returns:
        dist_panel (np.ndarray): shape of pure, except the last axis, 
            which has length 4 (for the number of distances computed). 
    """
    # Make axis 0 the axis indexing distance metrics, to begin with
    # And move it to the last axis before returning
    dist_array = np.zeros([4] + list(mixes.shape[:-1]))
    # No need to add axes to target vector; if it is 1d, it is broadcasted
    # along the last axis of mixes, which indexes elements of each vector. 
    dist_array[0] = l2_norm(target - mixes)
    dist_array[1] = l1_norm(target - mixes)
    dist_array[2] = linf_norm(target - mixes)
    dist_array[3] = cosine_dist(target, mixes)
    
    return np.moveaxis(dist_array, 0, -1)

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)
plt.rcParams["axes.facecolor"] = (1,1,1,0)  # transparent background

In [ ]:
models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "optimal": "Optimal", 
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "ideal": "xkcd:light green",
    "optimal": "xkcd:powder blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Initialization

In [ ]:
# Initialize common simulation parameters
n_dimensions = 300  # Half the real number for faster simulations
n_components = 3  # Number of background odors

inhib_rates = [0.00025, 0.00005]  # alpha, beta

# Simulation duration
duration = 320000.0
deltat = 1.0
skp = 200

# Common model options
activ_function = "identity"  #"ReLU"

# Background process
update_fct = update_thirdmoment_kinputs

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0xb1942912c9f11e45a71cbde601106048)
back_components = np.zeros([n_components, n_dimensions])
for i in range(n_components):
    back_components[i] = generate_odorant(n_dimensions, rgen_meta, lambda_in=0.1)
back_components = back_components / l2_norm(back_components).reshape(-1, 1)

# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Initial background vector and initial nu values
averages_nu = np.ones(n_components) / np.sqrt(n_components)
init_nu = np.zeros(n_components)
init_bkvec = averages_nu.dot(back_components)

## Compute the matrices in the Ornstein-Uhlenbeck update equation
# Update matrix for the mean term: 
# Exponential decay with time scale tau_nu over time deltat
tau_nu = 2.0  # Fluctuation time scale of the background nu_gammas (same for all)
update_mat_A = np.identity(n_components)*np.exp(-deltat/tau_nu)

# Steady-state covariance matrix
sigma2 = 0.09
correl_rho = 0.0
epsilon_nu = 0.2
steady_covmat = correl_rho * sigma2 * np.ones([n_components, n_components])  # Off-diagonals: rho
steady_covmat[np.eye(n_components, dtype=bool)] = sigma2  # diagonal: ones

# Cholesky decomposition of steady_covmat gives sqrt(tau/2) B
# Update matrix for the noise term: \sqrt(tau/2(1 - exp(-2*deltat/tau))) B
psi_mat = np.linalg.cholesky(steady_covmat)
update_mat_B = np.sqrt(1.0 - np.exp(-2.0*deltat/tau_nu)) * psi_mat

back_params = [update_mat_A, update_mat_B, back_components, averages_nu, epsilon_nu]
# nus are first in the list of initial background params
init_back_list = [init_nu, init_bkvec]

In [ ]:
# Analytical prediction, exact: need moments of nu. 
variance_nu3 = sigma2 + 2*(epsilon_nu*sigma2)**2
mean_nu3 = averages_nu[0] + epsilon_nu*sigma2
thirdmoment = 6*epsilon_nu*sigma2**2 + 8*(epsilon_nu*sigma2)**3
moments_conc = [mean_nu3, variance_nu3, thirdmoment]

## IBCM habituation
### IBCM simulation

In [ ]:
# IBCM model parameters
n_i_ibcm = 6  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.002  # 0.000001 = 1e-6
tau_avg_ibcm = 200
coupling_eta_ibcm = 0.5/n_i_ibcm
decay_relative_ibcm = 0.005
k_c2bar_avg = 1.0  # dummy
ssat_ibcm = 100.0 # dummy
lambd_ibcm = 0.725
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "linear", 
    "variant": "intrator", 
    "decay": False
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.2*rgen_meta.standard_normal(size=[n_i_ibcm, n_dimensions]) * lambd_ibcm

# Analytical prediction
cs_cn = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components-1, lambd=lambd_ibcm)
cs_cn = cs_cn[:2]
c_specif, c_nonspecif = cs_cn
lambda_max_pca = lambda_pca_equivalent(cs_cn, moments_conc, n_components, inhib_rates, verbose=True)

In [ ]:
# Run the IBCM simulations
sim_results = integrate_inhib_ibcm_network_options(
                init_synapses_ibcm, update_fct, init_back_list, 
                ibcm_rates, inhib_rates, back_params, duration, 
                deltat, seed=simul_seed, noisetype="normal",  
                skp=skp, **ibcm_options
)

(tser_ibcm, 
 nuser_ibcm, 
 bkvecser_ibcm, 
 mser_ibcm, 
 cbarser_ibcm, 
 thetaser_ibcm,
 wser_ibcm, 
 sser_ibcm) = sim_results

### IBCM habituation analysis

In [ ]:
# Calculate cgammas_bar and mbars
transient = 160000 // skp
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                    mser_ibcm, coupling_eta_ibcm, back_components)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction, exact: need moments of nu. 
variance_nu3 = sigma2 + 2*(epsilon_nu*sigma2)**2
mean_nu3 = averages_nu[0] + epsilon_nu*sigma2
thirdmoment = 6*epsilon_nu*sigma2**2 + 8*(epsilon_nu*sigma2)**3

# Compare to numerical values.
fullnuser3 = averages_nu.reshape(1, -1) + nuser_ibcm + epsilon_nu*nuser_ibcm**2
mean_nu3_sim = np.mean(fullnuser3)  # all odors i.i.d., can average over them. 
variance_nu3_sim = np.mean((fullnuser3 - mean_nu3_sim)**2)
thirdmoment_sim = np.mean((fullnuser3 - mean_nu3_sim)**3)
moments_conc = [mean_nu3, variance_nu3, thirdmoment]

# Analytical prediction
cs_cn = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components-1, lambd=lambd_ibcm)
cs_cn = cs_cn[:2]
c_specif, c_nonspecif = cs_cn

# Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
split_val = 1.0
cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
cgammas_bar_counts = {"above": int(np.sum(cbars_gamma_mean.flatten() > split_val)), 
                      "below": int(np.sum(cbars_gamma_mean.flatten() <= split_val))}
print(cgammas_bar_counts)

specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
print(specif_gammas)

# Analytical W
analytical_w = ibcm_fixedpoint_w_thirdmoment(inhib_rates, moments_conc, back_components, cs_cn, specif_gammas)

In [ ]:
# Plot the cbar2_avg term throughout
cbar2_avg_ser = moving_average(cbarser_ibcm*cbarser_ibcm, kernelsize=tau_avg_ibcm)
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
fig, ax = plt.subplots()
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm[:-tau_avg_ibcm], cbar2_avg_ser[:-tau_avg_ibcm, i], 
            color=neurons_cmap[i])
ax.set(xlabel="Time (x1000)", ylabel=r"$\bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=160000 // skp)
# Compare to exact analytical fixed point solution
ax.axhline(c_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = fullnuser3 - mean_nu3
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
plt.show()
plt.close()

# Check if each component has at least one neuron
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, sser_ibcm, skp=1)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
i_colors = sns.color_palette(n_colors=n_dimensions)
for j in range(n_i_ibcm):
    for i in range(n_dimensions):
        axes.flat[j].axhline(analytical_w[i, j], color=i_colors[i], ls="--", 
                             lw=0.5, alpha=0.7, zorder=-j*n_dimensions-i)
    #axes.flat[j].set_ylim([axes.flat[j].get_ylim()[0], np.amax(analytical_w[:, j])*1.1])
plt.show()
plt.close()

### Stability of average fixed points
Check the eigenvalues of the jacobian for one neuron, for every possible specificity. There are $2^{n_B}$ possibilities: choosing specific or not for each odor

In [ ]:
all_max_eigenvalues = ibcm_all_largest_eigenvalues(
    moments_conc, ibcm_rates, back_components, m3=1.0, cut=1e-16
)

In [ ]:
fig, ax = plt.subplots()
ibcm_specif_keys = list(all_max_eigenvalues.keys())
ibcm_eig_values = np.asarray([all_max_eigenvalues[a] for a in ibcm_specif_keys])
reals, imags = np.real(ibcm_eig_values), np.imag(ibcm_eig_values)
ibcm_eig_values_specif1 = np.asarray([len(s) == 1 for s in ibcm_specif_keys], dtype=bool)
highlights = ibcm_eig_values_specif1
ax.axvline(0.0, ls="--", color="k", lw=1.0)
ax.axhline(0.0, ls="--", color="k")
scaleup = 1e3
ax.plot(reals[highlights]*scaleup, imags[highlights]*scaleup, marker="*", mfc="b", mec="b", 
        ls="none", label="One odor", ms=8)
ax.plot(reals[~highlights]*scaleup, imags[~highlights]*scaleup, marker="o", mfc="k", mec="k", 
       ls="none", label="0 or 2+ odors", ms=6)
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.legend(title="Specificity")
ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{max}})$    ($\times 10^{-3}$)", 
      ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{max}})$     ($\times 10^{-3}$)")
fig.tight_layout()
plt.show()
plt.close()

## BioPCA simulation
### BioPCA habituation simulation

In [ ]:
# BioPCA model parameters
n_i_pca = n_components  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 0.00025  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.2
lambda_max_pca = lambda_pca_equivalent(cs_cn, moments_conc, n_components, inhib_rates, verbose=True)
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]


# Initial synaptic weights: small positive noise
init_synapses_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_meta.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_ifpsp_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="normal", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 sser_pca) = sim_results

### BioPCA simulation analysis

In [ ]:
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res

In [ ]:
from utils.statistics import principal_component_analysis
from modelfcts.checktools import compute_pca_meankept

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
fig.set_size_inches(fig.get_size_inches()[0], 3*2)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, snorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, sser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, snorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_pca, wser_pca, skp=10)
plt.show()
plt.close()

### Average background subtraction simulation

In [ ]:
# Average subtraction model parameters
avg_options = {"activ_fct": activ_function}

# Initial synaptic weights: dummy
init_synapses_avg = np.zeros([1, n_dimensions])

In [ ]:
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="normal", skp=skp, **avg_options
)
tser_avg, bkser_avg, bkvecser_avg, wser_avg, sser_avg = sim_results

### Ideal inhibition
The component parallel to the background is reduced to beta / (2*alpha + beta). 

In [ ]:
back_projector = find_projector(back_components.T)
ideal_factor = inhib_rates[1] / (2*inhib_rates[0] + inhib_rates[1])
sser_ideal = bkvecser_ibcm * ideal_factor
sser_orthogonal = np.zeros(bkvecser_ibcm.shape)

### Optimal W manifold learning

In [ ]:
# This matrix depends on new odor concentrations, so define them here
# New odor concentrations
new_test_concs = np.asarray([0.5, 1.0])
new_test_concs *= mean_nu3
n_new_concs = len(new_test_concs)

# Compute optimal W matrix for all new odors possible
dummy_rgen = np.random.default_rng(0x13d1faf1db367eed7498000c84dfd345)
new_odors_from_distrib = generate_odorant([int(1e5), n_dimensions], dummy_rgen, lambda_in=0.1)
new_odors_from_distrib /= l2_norm(new_odors_from_distrib)[:, None]
optimal_matrices = compute_optimal_matrices(back_components, new_odors_from_distrib, moments_conc, new_test_concs)

# Use the W matrix for the lowest concentration to inhibit the background
sser_optimal = bkvecser_ibcm - bkvecser_ibcm.dot(optimal_matrices[0].T)

## Model comparison for background inhibition

In [ ]:
snorm_series = {
    "ibcm": l2_norm(sser_ibcm), 
    "biopca": l2_norm(sser_pca), 
    "avgsub": l2_norm(sser_avg), 
    "none": l2_norm(bkvecser_ibcm), 
    "ideal": l2_norm(sser_ideal),
    "optimal": l2_norm(sser_optimal), 
    "orthogonal": l2_norm(sser_orthogonal)
}
std_options = dict(kernelsize=201, boundary="free")
mean_options = dict(kernelsize=201, boundary="free")
std_series = {
    a: np.sqrt(moving_var(snorm_series[a], **std_options)) for a in snorm_series
} 
mean_series = {
    a: moving_average(snorm_series[a], **mean_options) for a in snorm_series
}

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
axes = axes.flatten()
for model in std_series.keys():
    props = dict(label=model_nice_names[model], color=model_colors[model])
    axes[0].plot(tser_ibcm / 1000, mean_series[model], **props)
    axes[1].plot(tser_ibcm / 1000, std_series[model], **props)
snorm_string = r"$\|\vec{s}\|$"
axes[0].set_ylabel(r"PN activity norm, " + snorm_string)
axes[1].set(xlabel="Time (x1000 steps)", ylabel=r"Standard deviation " + snorm_string)
axes[0].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
fig.set_size_inches(4.5, 2.5*2)
plt.show()
plt.close()

## Model comparison for background tagging after habituation
We create a projection matrix, then compute the tag assigned to the background after inhibition by each habituation model, over time. Hopefully, only IBCM inhibits enough to see tags go to zero. 

In [ ]:
# Common parameters
n_kc = 1000 * n_dimensions // 25
projection_arguments = {
    "kc_sparsity": 0.05,
    "adapt_kc": True,
    "n_pn_per_kc": 3,
    "project_thresh_fact": 0.1
}
proj_mat = create_sparse_proj_mat(n_kc, n_dimensions, rgen_meta)

In [ ]:
# Computing background tag lengths at various time points for each model
sser_dict = {
    "ibcm": sser_ibcm, 
    "biopca": sser_pca, 
    "avgsub": sser_avg, 
    "none": bkvecser_ibcm, 
    "ideal": sser_ideal, 
    "optimal": sser_optimal, 
    "orthogonal": sser_orthogonal
}
tag_length_series = {a: np.zeros(tser_ibcm.shape[0]) for a in sser_dict.keys()}
for a in sser_dict.keys():
    for i in range(0, tag_length_series[a].shape[0]):
        if bkvecser_ibcm[i].max() > 0:
            tag = project_neural_tag(sser_dict[a][i], bkvecser_ibcm[i], 
                                 proj_mat, **projection_arguments)
        else:
            tag = (1,)*int(projection_arguments["kc_sparsity"]*n_kc)
        tag_length_series[a][i] = len(tag)
tag_length_series_smooth = {a: moving_average(tag_length_series[a], **mean_options)
                            for a in tag_length_series}

In [ ]:
fig, ax = plt.subplots()
for model in sser_dict.keys():
    props = dict(label=model_nice_names[model], color=model_colors[model])
    ax.plot(tser_ibcm / 1000, tag_length_series_smooth[model], **props)
snorm_string = r"$\|\vec{s}\|$"
ax.set_ylabel(r"Tag length, $\mathrm{card}(z)$")
ax.set_xlabel("Time (x1000 steps)")
ax.legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
plt.show()
plt.close()

## Model comparison for new odor recognition

In [ ]:
def find_snap_index(dt, skip, times):
    """ Find nearest multiple of dt*skip to each time in times """
    return np.around(times / (dt*skip)).astype(int)

In [ ]:
# Generate new odors, select test times, etc.
# New odors tested
n_new = 10
new_odors = generate_odorant([n_new, n_dimensions], rgen_meta, lambda_in=0.1)
new_odors /= l2_norm(new_odors)[:, None]

# Test times
n_test_times = 10
start_test_t = duration - n_test_times * 2000.0
test_times = np.linspace(start_test_t, duration, n_test_times)
test_times -= deltat*skp
test_idx = find_snap_index(deltat, skp, test_times)

# New odor concentrations, defined above

# Background samples, indexed [time, sample, n_orn]
n_back_samples = 10
# sample_ss_distrib_thirdmoment(means_nu, covmat_nu, epsil, size=1, rgen=None)
conc_samples = sample_ss_distrib_thirdmoment(
                    averages_nu, steady_covmat, epsil=epsilon_nu, 
                    size=n_test_times*(n_back_samples-1), rgen=rgen_meta
                )  # Shaped [sample, component]
back_samples = conc_samples.dot(back_components)
back_samples = back_samples.reshape([n_test_times, n_back_samples-1, -1])
back_samples = np.concatenate([bkvecser_ibcm[test_idx, None, :], back_samples], axis=1)

# Containers for s vectors of each model
mixture_yvecs = {a: np.zeros([n_new, n_test_times,  n_new_concs,  
                    n_back_samples, n_dimensions]) for a in sser_dict.keys()}
mixture_tags = {a: SparseNDArray((n_new, n_test_times, n_new_concs,
                    n_back_samples, n_kc), dtype=bool) for a in sser_dict.keys()}
new_odor_tags = sparse.lil_array((n_new, n_kc), dtype=bool)
jaccard_scores = {a: np.zeros([n_new, n_test_times, n_new_concs,  n_back_samples]) 
                  for a in sser_dict.keys()}

In [ ]:
# Compute ideal reduction factor for each concentration
ideal_factors = [compute_ideal_factor(c, moments_conc[:2], [n_components, n_dimensions], 
                    generate_odorant, (dummy_rgen,), reps=200) for c in new_test_concs]
# Approximate ideal factors:
#ideal_factors = [0.1, 0.18]
print("Finished computing ideal factors")
for i in range(n_new):
    # Compute neural tag of the new odor alone, without inhibition
    new_tag = project_neural_tag(
                    new_odors[i], new_odors[i],
                    proj_mat, **projection_arguments
                )
    new_odor_tags[i, list(new_tag)] = True
    # Parallel and orthogonal components
    x_new_par = find_parallel_component(new_odors[i], 
                        back_components, back_projector)
    x_new_ort = new_odors[i] - x_new_par
    # Now, loop over snapshots, mix the new odor with the back samples,
    # compute the PN response at each test concentration,
    # compute tags too, and save results
    for j in range(n_test_times):
        jj = test_idx[j]
        for k in range(n_new_concs):
            mixtures = (back_samples[j]
                + new_test_concs[k] * new_odors[i])
            # odors, mlx, wmat, 
            # Compute for each model
            mixture_yvecs["ibcm"][i, j, k] = ibcm_respond_new_odors(
                mixtures, mser_ibcm[jj], wser_ibcm[jj], 
                ibcm_rates, options=ibcm_options
            )
            mixture_yvecs["biopca"][i, j, k] = biopca_respond_new_odors(
                mixtures, [mser_pca[jj], lser_pca[jj], xser_pca[jj]], 
                wser_pca[jj], biopca_rates, options=pca_options
            )
            mixture_yvecs["avgsub"][i, j, k] = average_sub_respond_new_odors(
                mixtures, wser_avg[jj], options=avg_options
            )
            mixture_yvecs["none"][i, j, k] = mixtures
            mixture_yvecs["ideal"][i, j, k] = ideal_linear_inhibitor(
                x_new_par, x_new_ort, mixtures, new_test_concs[k], 
                ideal_factors[k], **avg_options
            )
            mixture_yvecs["optimal"][i, j, k] = mixtures - mixtures.dot(optimal_matrices[k].T)
            mixture_yvecs["orthogonal"][i, j, k] = new_test_concs[k] * x_new_ort
            for l in range(n_back_samples):
                for mod in mixture_yvecs.keys():
                    mix_tag = project_neural_tag(
                        mixture_yvecs[mod][i, j, k, l], mixtures[l],
                        proj_mat, **projection_arguments
                    )
                    try:
                        mixture_tags[mod][i, j, k, l, list(mix_tag)] = True
                    except ValueError as e:
                        print(mix_tag)
                        print(mixture_yvecs[mod][i, j, k, l])
                        print(proj_mat.dot(mixture_yvecs[mod][i, j, k, l]))
                        raise e
                    jaccard_scores[mod][i, j, k, l] = jaccard(mix_tag, new_tag)

In [ ]:
ideal_factors

In [ ]:
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "avgsub", "biopca", "ibcm", "optimal", "orthogonal",  "ideal"]
for m in models:  # Plot IBCM last
    all_jacs = jaccard_scores[m]
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_jacs[:, :, i, :].flatten(),
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            np.median(all_jacs[:, :, i, :]), ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    ax = axes[i]
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel("Jaccard similarity (higher is better)")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("figures/detection/compare_models_onerun_non-gaussian_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
# Distance to new odor
# Plot model histogram results
# One plot per new odor concentration
fig, axes = plt.subplots(1, n_new_concs, sharex=True)
fig.set_size_inches(9.5, 4)
axes = axes.flatten()
models = ["none", "avgsub", "biopca", "ibcm", "optimal", "orthogonal"]#,  "ideal"]
all_medians = []
for m in models:  # Plot IBCM last
    all_distances = (mixture_yvecs[m] 
         - new_test_concs[None, None, :, None, None]*new_odors[:, None, None, None, :])
    all_norms = l2_norm(all_distances.reshape(-1, n_dimensions))
    all_medians.append(np.median(all_norms))
    for i in range(n_new_concs):
        hist_outline(
            axes[i], all_norms,
            bins="doane", density=True, label=model_nice_names.get(m, m),
            color=model_colors.get(m), alpha=1.0
        )
        axes[i].axvline(
            all_medians[-1], ls="--",
            color=model_colors.get(m)
        )
# Labeling the graphs, etc.
for i in range(n_new_concs):
    axes[i].set_xlim([0.0, 2.0*max(all_medians)])
    axes[i].set_title("New conc. = {:.1f}".format(new_test_concs[i]))
    axes[i].set_xlabel(r"Distance to new odor, $\|\vec{s} - \vec{x}_{\mathrm{new}}\|$")
    axes[i].set_ylabel("Probability density")
axes[1].legend(loc="upper left", bbox_to_anchor=(1.0, 1.0), frameon=False)
fig.tight_layout()
#fig.savefig("figures/detection/compare_models_onerun_non-gaussian_snorm_{}.pdf".format(activ_fct),
#            transparent=True, bbox_inches="tight")

plt.show()
plt.close()

In [ ]:
# We also want an unskipped nu time series for the figure, run a dummy simulation for its background
sim_results = integrate_inhib_average_sub_skip(
                init_synapses_avg, update_fct, init_back_list, 
                [], inhib_rates, back_params, duration, deltat,
                seed=simul_seed, noisetype="normal", skp=1, **avg_options
)
_, nuser_noskp, _, _, _ = sim_results

## Miscellaneous analyses

In [ ]:
ideal_factors

In [ ]:
inhib_rates[1] / (inhib_rates[1] + inhib_rates[0]*moments_conc[1]*(cs_cn[1]-cs_cn[0])**2)

In [ ]:
print("Relative norm of the WLM projection of the parallel component:", 
      np.sqrt(np.sum((wser_ibcm[-1].dot(mser_ibcm[-1]).dot(x_new_par))**2) / np.sum(x_new_par**2)))
print("Relative norm of the WLM projection of the orthogonal component:", 
      np.sqrt(np.sum((wser_ibcm[-1].dot(mser_ibcm[-1]).dot(x_new_ort))**2) / np.sum(x_new_ort**2)))


In [ ]:
plt.bar(np.arange(50), mixture_yvecs["ibcm"][0, 0, 0, 0, :50], width=0.5, label="IBCM")
plt.bar(np.arange(50)+0.5, mixture_yvecs["orthogonal"][0, 0, 0, 0, :50], width=0.5, label="Orthogonal")
plt.legend()
plt.show()
plt.close()